In [1]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd

In [2]:
# Twitter認証情報
consumer_key        = 'hophVgLGWBfTJLaqD9E0pSyO1'
consumer_secret     = '67gkgbvLlfIIr8j1nSW1VCss1OVUK66vs00dAXqZ7VWYUTKGBX'
access_token        = '1611281223095300102-KrgKZvwMVrjh4QuX2q1uDKwHeJqUzl'
access_token_secret = '9dbM9QTVa1wxAZnmOtwLF1u84wSQVbrWlDoA3G66TS48s'


In [17]:
#Twitterの認証
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [18]:
#日本のWOEID
woeid = 23424856
#トレンド一覧取得
trends = api.get_place_trends(woeid)

In [19]:
trends[0]['trends']
trends_list = [trend['name'] for trend in trends[0]['trends']]

In [25]:
print(len(trends_list))

50


In [20]:
trends_list

['Twitter連携',
 'API有料化',
 'ツインテールの日',
 'ポイピク',
 'Twilog',
 'べったー',
 'ココフォリア',
 'ツイッター連携',
 'じゃっさん',
 'Twitter認証',
 'ファイズ',
 'プリキュア',
 'Google連携',
 'マジックミラー号',
 '#鬼滅の刃',
 '人身事故',
 'キュワワー',
 'パモさん',
 'プライベッター',
 '#スシローを救いたい',
 'ローション相撲',
 'メビウス',
 '恵方巻き',
 '無料提供',
 'Twitterアカウント',
 'グングニル',
 'OAuth',
 'しゅうまい',
 'イヴェルカーナ',
 'メアド登録',
 '連携アプリ',
 '連携サービス',
 'マストドン',
 'くら寿司',
 'ヒストリーメイカー',
 '#刀鍛冶の里編',
 '急病の男児',
 'ヤフーとLINE',
 '救急車の搬送妨害疑い',
 '会社役員逮捕',
 'ログイン方法',
 '#karintalk',
 'Twitter以外',
 '個人サイト',
 'ログイン連携',
 'トニコン',
 '#祝ツキヨミ7500万回',
 'ログイン系',
 'INI渡韓',
 'マイクロビキニ']

In [6]:
#　”wait_on_rate_limit = True”　利用制限にひっかかた時に必要時間待機する
api=tweepy.API(auth,wait_on_rate_limit=True)

In [21]:
# 検索条件の設定
search_word = 'python'
#何件のツイートを取得するか
item_number = 10000

In [22]:
#検索条件を元にツイートを抽出
tweets = tweepy.Cursor(api.search_tweets,q=search_word, tweet_mode='extended',result_type="mixed",lang='ja').items(item_number)

In [23]:
#関数:　UTCをJSTに変換する
def change_time_JST(u_time):
    #イギリスのtimezoneを設定するために再定義する
    utc_time = datetime(u_time.year, u_time.month,u_time.day, \
    u_time.hour,u_time.minute,u_time.second, tzinfo=timezone.utc)
    #タイムゾーンを日本時刻に変換
    jst_time = utc_time.astimezone(pytz.timezone("Asia/Tokyo"))
    # 文字列で返す
    str_time = jst_time.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time

In [24]:
#抽出したデータから必要な情報を取り出す
#取得したツイートを一つずつ取り出して必要な情報をtweet_dataに格納する
tw_data = []

for tweet in tweets:
    #ツイート時刻とユーザのアカウント作成時刻を日本時刻にする
    tweet_time = change_time_JST(tweet.created_at)
    create_account_time = change_time_JST(tweet.user.created_at)
    #tweet_dataの配列に取得したい情報を入れていく
    tw_data.append([
        tweet.id,
        tweet_time,
        tweet.full_text,
        tweet.favorite_count, 
        tweet.retweet_count, 
        tweet.user.id, 
        tweet.user.screen_name,
        tweet.user.name,
        tweet.user.description,
        tweet.user.friends_count,
        tweet.user.followers_count,
        create_account_time,
        tweet.user.following,
        tweet.user.profile_image_url,
        tweet.user.profile_background_image_url,
        tweet.user.url
                       ])

#取り出したデータをpandasのDataFrameに変換
#CSVファイルに出力するときの列の名前を定義
labels=[
    'ツイートID',
    'ツイート時刻',
    'ツイート本文',
    'いいね数',
    'リツイート数',
    'ID',
    'ユーザー名',
    'アカウント名',
    '自己紹介文',
    'フォロー数',
    'フォロワー数',
    'アカウント作成日時',
    '自分のフォロー状況',
    'アイコン画像URL',
    'ヘッダー画像URL',
    'WEBサイト'
    ]

#tw_dataのリストをpandasのDataFrameに変換
df = pd.DataFrame(tw_data,columns=labels)

#CSVファイルに出力する
#CSVファイルの名前を決める
file_name='tw_data.csv'

#CSVファイルを出力する
df.to_csv(file_name,encoding='utf-8-sig',index=False)

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded